# Grover's Algorithm: An Intuitive and Complete Explanation

## 1. Core Objective: Quadratic Speedup

In **classical computing**, searching for a specific item in an *unsorted* database of $N$ items requires checking items one by one. In the worst case, this takes $O(N)$ operations.

**Grover's algorithm** provides a **quadratic speedup** by finding the desired item using only $O(\sqrt{N})$ oracle queries.

This speedup is achieved through **amplitude amplification**: the probability amplitude of the correct (winner) state is increased, while the amplitudes of all incorrect (loser) states are decreased.

---

## 2. Initialization and Equal Superposition

### Database Encoding

We use $n$ qubits to represent a database of size  
$$N = 2^n.$$

Each computational basis state $|x\rangle$ corresponds to one database item.

### Creating the Equal Superposition

All qubits are initialized to $|0\rangle^{\otimes n}$. Applying a Hadamard gate to each qubit produces an equal superposition over all possible inputs:
$$
|\psi_{\text{in}}\rangle
= H^{\otimes n} |0\rangle^{\otimes n}
= \frac{1}{\sqrt{N}} \sum_{x=0}^{N-1} |x\rangle.
$$

At this stage, every item has the same probability $1/N$ of being measured.

---

## 3. Target Qubit and Phase Kickback

To mark the correct item **without disturbing probabilities directly**, we introduce one auxiliary **target qubit**.

### Preparing the Target Qubit

The target qubit is initialized in the state:
$$
|-\rangle = \frac{|0\rangle - |1\rangle}{\sqrt{2}}.
$$

The combined system state becomes:
$$
|\psi_1\rangle = |\psi_{\text{in}}\rangle \otimes |-\rangle.
$$

This special state enables **phase kickback**, a key quantum trick used by the oracle.

---

## 4. The Oracle Transformation ($U_f$)

### Defining the Oracle Function

The oracle encodes the solution $k$ using a Boolean function:
$$
f(x) =
\begin{cases}
1, & x = k \quad \text{(winner)} \\
0, & x \neq k \quad \text{(losers)}
\end{cases}
$$

### Oracle Action

The oracle unitary acts as:
$$
U_f |x\rangle |y\rangle = |x\rangle |y \oplus f(x)\rangle.
$$

When the target qubit is in the $|-\rangle$ state, flipping it introduces a **phase change** on the input state. This gives:
$$
U_f |x\rangle = (-1)^{f(x)} |x\rangle.
$$

### Effect of the Oracle

Only the winner state acquires a negative sign:
$$
|\psi_2\rangle
= \frac{1}{\sqrt{N}} \sum_{x \neq k} |x\rangle
- \frac{1}{\sqrt{N}} |k\rangle.
$$

Importantly, **probabilities remain unchanged**, but the *phase* of the correct state is flipped.

---

## 5. Geometric Interpretation: A 2D State Space

Although the system lives in a $2^n$-dimensional space, Grover's algorithm operates entirely in a **two-dimensional subspace**.


![Grover's Algorithm Geometry](fig/grover_geometry.png)

### Basis Vectors

We define two orthonormal states:

- **Winner state**: $|k\rangle$
- **Non-winner superposition**:
  $$
  |c\rangle = \frac{1}{\sqrt{N-1}} \sum_{x \neq k} |x\rangle
  $$

### Decomposition of the Initial State

The equal superposition can be written as:
$$
|\psi_{\text{in}}\rangle = \sin \theta \, |k\rangle + \cos \theta \, |c\rangle,
$$
where
$$
\sin \theta = \frac{1}{\sqrt{N}}.
$$

Initially, the state vector is very close to $|c\rangle$ and far from $|k\rangle$.

---

## 6. The Grover Iterator: Two Reflections

One **Grover iteration** consists of two reflections that together form a rotation.


---

### Step A: Oracle Reflection (About $|c\rangle$)

The oracle flips the sign of the $|k\rangle$ component:
$$
|\psi_{\text{mid}}\rangle = \cos \theta \, |c\rangle - \sin \theta \, |k\rangle.
$$

Geometrically, this reflects the state across the $|c\rangle$ axis.

---

### Step B: Diffusion Operator (About $|\psi_{\text{eq}}\rangle$)

We have the state after the oracle:
$$
|\psi_{\text{mid}}\rangle
= \cos\theta \, |c\rangle - \sin\theta \, |k\rangle
=
\begin{bmatrix}
\cos\theta \\
-\sin\theta
\end{bmatrix}.
$$

The diffusion operator in the $\{|c\rangle,|k\rangle\}$ basis is:
$$
U_{\psi_{\text{eq}}}
=
\begin{bmatrix}
\cos 2\theta & \sin 2\theta \\
\sin 2\theta & -\cos 2\theta
\end{bmatrix}.
$$

Apply the operator:
$$
|\psi_{\text{out}}\rangle
=
\begin{bmatrix}
\cos 2\theta & \sin 2\theta \\
\sin 2\theta & -\cos 2\theta
\end{bmatrix}
\begin{bmatrix}
\cos\theta \\
-\sin\theta
\end{bmatrix}.
$$

Upper component:
$$
\cos 2\theta \cos\theta - \sin 2\theta \sin\theta
= \cos(2\theta+\theta)
= \cos 3\theta.
$$

Lower component:
$$
\sin 2\theta \cos\theta + \cos 2\theta \sin\theta
= \sin(2\theta+\theta)
= \sin 3\theta.
$$

Therefore:
$$
|\psi_{\text{out}}\rangle
=
\begin{bmatrix}
\cos 3\theta \\
\sin 3\theta
\end{bmatrix}
=
\cos 3\theta \, |c\rangle + \sin 3\theta \, |k\rangle.
$$


---

### Key Result

Each Grover iteration **rotates the state vector by an angle $2\theta$ toward $|k\rangle$**, steadily increasing the probability of measuring the correct answer.

---

## 7. Example Case: $N = 4$

For a 2-qubit database:

- $N = 4$
- $\sin \theta = \frac{1}{2} \Rightarrow \theta = 30^\circ$

After **one Grover iteration**:
$$
3\theta = 90^\circ
$$

The state aligns exactly with $|k\rangle$, giving:
$$
P(\text{success}) = \sin^2(90^\circ) = 1.
$$

Thus, the correct item is found with certainty in **one iteration**.

---

## 8. Constructing the Diffusion Operator in Circuits

The equal superposition state used in Grover’s algorithm is
$$
|\psi_{\text{eq}}\rangle = H^{\otimes n} |0\rangle^{\otimes n}.
$$

The diffusion operator is defined as a reflection about this state:
$$
U_{\psi_{\text{eq}}} = 2|\psi_{\text{eq}}\rangle\langle\psi_{\text{eq}}| - I.
$$

Substituting $|\psi_{\text{eq}}\rangle = H^{\otimes n} |0\rangle^{\otimes n}$ gives
$$
U_{\psi_{\text{eq}}}
=
H^{\otimes n}
\left( 2|0\rangle^{\otimes n}\langle 0|^{\otimes n} - I \right)
H^{\otimes n}.
$$

The inner operator $\left( 2|0\rangle^{\otimes n}\langle 0|^{\otimes n} - I \right)$ applies a phase transformation:
$$
|0\rangle^{\otimes n} \rightarrow |0\rangle^{\otimes n}, \qquad
|x\rangle \rightarrow -|x\rangle \;\; \text{for } x \neq 0.
$$

Physically, this means only the all-zero state keeps its phase, while every other basis state acquires a phase of $-1$.
Conjugating this operation by $H^{\otimes n}$ maps the phase flip back to a reflection about the equal superposition, which is exactly the diffusion step of Grover’s algorithm.

In a quantum circuit, the phase flip on $|0\rangle^{\otimes n}$ is implemented by:
applying $X$ gates to convert $|0\rangle^{\otimes n}$ into $|1\rangle^{\otimes n}$,
applying a multi-controlled $Z$ gate to flip the phase of $|1\rangle^{\otimes n}$,
and then applying $X$ gates again to restore the original basis.


---

## Complete Algorithm Summary

1. **Initialize**  
   - $n$ input qubits in $H^{\otimes n}|0\rangle^{\otimes n}$  
   - One target qubit in $|-\rangle$

2. **Repeat $m$ times**  
   - Oracle $U_f$: flips the phase of the winner  
   - Diffusion $U_{\psi_{\text{eq}}}$: amplifies the winner amplitude

3. **Measure**  
   - Measure the input qubits to obtain $k$

The optimal number of Grover iterations is approximately:
$$
m \approx \frac{\pi}{4}\sqrt{N}.
$$

This completes the explanation of how Grover's algorithm achieves quadratic speedup through elegant geometric rotations in quantum state space.

# Algorithm code explained

We analyze the 2-qubit Grover search implemented in the code for the marked element
$$
k = |01\rangle,
$$
with database size
$$
N = 2^2 = 4.
$$
The computational basis is
$$
\{|00\rangle, |01\rangle, |10\rangle, |11\rangle\}.
$$
The explanation below follows **exactly the logic of the gates used in the code**, but is presented using theory and mathematics only.

---

### 1. Input Register Initialization (Hadamard Gates)

Both input qubits start in the state $|00\rangle$.
Applying a Hadamard gate $H$ to **each input qubit** creates an equal superposition:
$$
H^{\otimes 2}|00\rangle
= \frac{1}{2} \left(
|00\rangle + |01\rangle + |10\rangle + |11\rangle
\right).
$$

The Hadamard gates are responsible for spreading probability amplitude uniformly across all possible database entries.

---

### 2. Target Qubit Preparation (X + H Gates)

The target (ancilla) qubit is prepared using:
- an $X$ gate
- followed by a Hadamard gate $H$

This produces the state
$$
|-\rangle = H X |0\rangle = \frac{|0\rangle - |1\rangle}{\sqrt{2}}.
$$

This specific state is crucial because it enables **phase kickback**: any controlled flip of the target qubit results in a phase flip on the input state.

---

### 3. Oracle Construction for the Secret State $|01\rangle$

The oracle marks the solution by flipping its phase. The following gates are used.

#### (a) X Gates on Zero Bits

The secret string is $01$.
To detect this pattern:
- the first qubit corresponds to bit $0$, so an $X$ gate is applied
- the second qubit corresponds to bit $1$, so no gate is applied

This temporarily maps:
$$
|01\rangle \longrightarrow |11\rangle.
$$

All other basis states are mapped to states containing at least one $0$.

---

#### (b) Multi-Controlled Operation (Toffoli Gate)

A Toffoli gate (controlled-controlled-NOT) is applied using both input qubits as controls and the target qubit as the target.

At this stage, the oracle needs a way to **recognize exactly one input state** and mark it, which is done using a Toffoli gate.

The Toffoli gate is a controlled-controlled-NOT: it acts like a classical **AND check**. It flips the target qubit **only if all control qubits are in state $|1\rangle$**. In our 2-qubit example:

- The first qubit corresponds to the first bit of the secret element, the second qubit to the second bit.
- Before applying the Toffoli, the oracle has applied $X$ gates on qubits corresponding to 0s in the secret element.  
  For the secret $|01\rangle$, the first qubit is flipped using $X$, so the state $|01\rangle$ temporarily becomes $|11\rangle$.
- All other basis states now contain at least one $0$, so they **do not trigger** the Toffoli.

Now the Toffoli gate is applied:

- If the input is $|11\rangle$ (originally $|01\rangle$), the target qubit is flipped.
- If the input is anything else, the target qubit is unchanged.

The target qubit is prepared in the state
$$
|-\rangle = \frac{|0\rangle - |1\rangle}{\sqrt{2}}.
$$
Flipping this state with a NOT ($X$) gate gives
$$
X|-\rangle = -|-\rangle,
$$
as can be verified:
$$
X|-\rangle = X\frac{|0\rangle - |1\rangle}{\sqrt{2}} = \frac{|1\rangle - |0\rangle}{\sqrt{2}} = -\frac{|0\rangle - |1\rangle}{\sqrt{2}} = -|-\rangle.
$$

**Explanation:** The target qubit itself does not change its form; instead, the minus sign is **kicked back to the input qubits**. This is the essence of **phase kickback**: the input state corresponding to the secret element now has a negative amplitude.

As a result:

- The secret element $|01\rangle$ acquires a phase of $-1$.
- All other input states remain unchanged.

This means the Toffoli gate does **not affect measurement probabilities** yet; it only **marks the correct state by flipping its phase**, which is exactly what Grover’s algorithm needs. Later, the diffusion operator will use this phase information to amplify the probability of measuring the marked state.


---

#### (c) X Gates to Restore the Input Basis

The same $X$ gate applied earlier is applied again to the first qubit.
This restores all basis states to their original computational values while preserving the phase change.

After the oracle, the input register is:
$$
\frac{1}{2}\left(
|00\rangle
- |01\rangle
+ |10\rangle
+ |11\rangle
\right).
$$

Only the amplitude of $|01\rangle$ has changed sign.

---

### 4. Diffusion Operator (Reflection About the Mean)

The diffusion operator, also called **inversion about the mean**, is applied only to the input qubits. Its purpose is to amplify the probability of the marked state after the oracle has flipped its phase.  

For 2 qubits, the diffusion operator is implemented with the following gate sequence:

---

#### a. Full Operator Concept

Mathematically, the diffusion operator is:
$$
U_{\psi_{\text{eq}}} = 2|\psi_{\text{eq}}\rangle\langle\psi_{\text{eq}}| - I,
$$
where
$$
|\psi_{\text{eq}}\rangle = \frac{1}{2}(|00\rangle + |01\rangle + |10\rangle + |11\rangle)
$$
is the equal superposition of all basis states.  

The action of this operator is:
- For any amplitude $a_x$, it maps $a_x \rightarrow 2\bar{a} - a_x$, where $\bar{a}$ is the average amplitude.
- This reflects the state vector across $|\psi_{\text{eq}}\rangle$ in Hilbert space.

---

#### b. Step-by-Step Gate Implementation

**(i) Hadamard Gates ($H^{\otimes 2}$)**

- Apply a Hadamard gate to each input qubit.
- This transforms the computational basis states into the superposition basis, effectively moving from $|00\rangle, |01\rangle, |10\rangle, |11\rangle$ to a combination of $|\pm\rangle$ states.
- This is the first part of preparing the reflection about the mean.

**(ii) X Gates on All Input Qubits**

- Apply an $X$ gate to each input qubit.
- This maps the state $|00\rangle \longrightarrow |11\rangle$, so that the "all-zero" state (which is used to implement the phase flip) is now represented as "all-one."
- All other basis states are similarly flipped: $|01\rangle \rightarrow |10\rangle$, $|10\rangle \rightarrow |01\rangle$, $|11\rangle \rightarrow |00\rangle$.
- Intuition: we want a **controlled-phase flip on the $|00\rangle$ state**, so flipping it to $|11\rangle$ allows us to implement it using a Toffoli/CNOT sequence.

**(iii) Controlled Phase Flip (H–CNOT–H)**

- This step implements a **conditional $Z$ gate** on the state $|11\rangle$ (which represents the original $|00\rangle$):
  1. Apply $H$ to the second qubit.
  2. Apply a CNOT with the first qubit as control and second as target.
  3. Apply $H$ again to the second qubit.
- This sequence flips the phase of $|11\rangle$ by $-1$ without affecting other states.
- Explanation: flipping the phase of this "all-one" state is equivalent to flipping the phase of the "all-zero" state before the $X$ gates. This is the **core of inversion about the mean**.

**(iv) X Gates Again**

- Apply $X$ gates to each qubit again to restore the computational basis.
- Now $|11\rangle$ returns to $|00\rangle$, and all other states are restored.
- The phase flip remains only on the original $|00\rangle$ amplitude.

**(v) Final Hadamard Gates ($H^{\otimes 2}$)**

- Apply Hadamard gates to each qubit again.
- This maps the reflection from the superposition basis back into the computational basis.
- Result: the amplitudes of all states are updated according to
$$
a_x \longrightarrow 2\bar{a} - a_x
$$
- Geometrically, the state vector is **reflected about the equal superposition axis**, increasing the amplitude of the marked state and decreasing the others.

---

### 5. Amplitude Evolution (Inversion About the Mean)

Before diffusion:
$$
a_{01} = -\frac{1}{2},
\quad
a_{\text{others}} = \frac{1}{2}.
$$

The mean amplitude is:
$$
\bar{a} = \frac{1}{4}.
$$

Applying inversion about the mean:
$$
a_x \rightarrow 2\bar{a} - a_x
$$

gives:
$$
a_{01} = 1,
\quad
a_{\text{others}} = 0.
$$

Thus, after diffusion:
$$
|\psi_{\text{out}}\rangle = |01\rangle.
$$

---

### 6. Measurement

Finally, measuring the input qubits yields:
$$
|01\rangle
$$
with probability $1$.

---

### Final Insight

Every gate in the circuit has a clear role:

- $H$ gates create͟ create and undo superpositions
- $X$ gates align the desired state with $|11\rangle$
- the Toffoli gate marks the solution using phase kickback
- the CNOT-based controlled-$Z$ performs inversion about the mean

Together, these gates implement **one full Grover iteration**, which is sufficient to deterministically find $|01\rangle$ in a 2-qubit search space.


# Cirq Implementation

In [4]:
import cirq
import numpy as np


def oracle(input_qubits, target_qubit, circuit, secret_element='01'):
    print(f"Element to be searched: {secret_element}")

    # Flip qubits corresponding to 0s in the secret element
    for i, bit in enumerate(secret_element):
        if int(bit) == 0:
            circuit.append(cirq.X(input_qubits[i]))

    # Conditional NOT (multi-controlled operation)
    circuit.append(cirq.TOFFOLI(*input_qubits, target_qubit))

    # Revert the flipped qubits
    for i, bit in enumerate(secret_element):
        if int(bit) == 0:
            circuit.append(cirq.X(input_qubits[i]))

    return circuit


def grovers_algorithm(num_qubits=2, copies=1000):
    # Define input and target qubits
    input_qubits = [cirq.LineQubit(i) for i in range(num_qubits)]
    target_qubit = cirq.LineQubit(num_qubits)

    # Define quantum circuit
    circuit = cirq.Circuit()

    # Create equal superposition
    circuit.append(cirq.H.on_each(*input_qubits))

    # Prepare target qubit in |-> state
    circuit.append(cirq.X(target_qubit))
    circuit.append(cirq.H(target_qubit))

    # Apply oracle
    circuit = oracle(input_qubits, target_qubit, circuit)

    # Grover diffusion operator
    circuit.append(cirq.H.on_each(*input_qubits))
    circuit.append(cirq.X.on_each(*input_qubits))
    circuit.append(cirq.H(input_qubits[1]))
    circuit.append(cirq.CNOT(input_qubits[0], input_qubits[1]))
    circuit.append(cirq.H(input_qubits[1]))
    circuit.append(cirq.X.on_each(*input_qubits))
    circuit.append(cirq.H.on_each(*input_qubits))

    # Measurement
    circuit.append(cirq.measure(*input_qubits, key='Z'))

    print("Grover's algorithm circuit:")
    print(circuit)

    # Simulation
    sim = cirq.Simulator()
    result = sim.run(circuit, repetitions=copies)

    # Histogram output
    out = result.histogram(key='Z')
    out_result = {}

    for k in out.keys():
        new_key = format(k, 'b')
        if len(new_key) < num_qubits:
            new_key = '0' * (num_qubits - len(new_key)) + new_key
        out_result[new_key] = out[k]

    print("Measurement results:")
    print(out_result)


if __name__ == '__main__':
    grovers_algorithm(2)


Element to be searched: 01
Grover's algorithm circuit:
0: ───H───X───@───X───H───X───@───X───H───────M('Z')───
              │               │               │
1: ───H───────@───H───X───H───X───H───X───H───M────────
              │
2: ───X───H───X────────────────────────────────────────
Measurement results:
{'01': 1000}


# Qiskit Implementation

In [7]:
from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram

def oracle(circuit, input_qubits, target_qubit, secret_element='01'):
    print(f"Element to be searched: {secret_element}")

    # Flip qubits corresponding to 0s in the secret element
    for i, bit in enumerate(secret_element):
        if int(bit) == 0:
            circuit.x(input_qubits[i])

    # Multi-controlled NOT (Toffoli) on the target qubit
    circuit.mcx(input_qubits, target_qubit)

    # Revert the flipped qubits
    for i, bit in enumerate(secret_element):
        if int(bit) == 0:
            circuit.x(input_qubits[i])

    return circuit

def grovers_algorithm(num_qubits=2, shots=1000):
    # Define input and target qubits
    circuit = QuantumCircuit(num_qubits + 1, num_qubits)  # last qubit is target
    input_qubits = list(range(num_qubits))
    target_qubit = num_qubits

    # Create equal superposition on input qubits
    for q in input_qubits:
        circuit.h(q)

    # Prepare target qubit in |-> state
    circuit.x(target_qubit)
    circuit.h(target_qubit)

    # Apply oracle
    circuit = oracle(circuit, input_qubits, target_qubit)

    # Grover diffusion operator
    # 1. Hadamard on input qubits
    for q in input_qubits:
        circuit.h(q)
    # 2. X gates on input qubits
    for q in input_qubits:
        circuit.x(q)
    # 3. Controlled-Z using H-CNOT-H trick on 2-qubit case
    circuit.h(input_qubits[1])
    circuit.cx(input_qubits[0], input_qubits[1])
    circuit.h(input_qubits[1])
    # 4. X gates again
    for q in input_qubits:
        circuit.x(q)
    # 5. Hadamard on input qubits
    for q in input_qubits:
        circuit.h(q)

    # Measurement
    for i, q in enumerate(input_qubits):
        circuit.measure(q, i)

    print("Grover's algorithm circuit:")
    print(circuit)

    # Simulation
    sim = AerSimulator()
    result = sim.run(circuit, shots=shots).result()
    counts = result.get_counts()

    # Reverse bitstrings for human-readable order
    counts_reversed = {k[::-1]: v for k, v in counts.items()}

    print("Measurement results (reversed):")
    print(counts_reversed)

    return counts_reversed

if __name__ == "__main__":
    grovers_algorithm(2)


Element to be searched: 01
Grover's algorithm circuit:
     ┌───┐┌───┐     ┌───┐┌───┐┌───┐     ┌───┐┌───┐     ┌─┐   
q_0: ┤ H ├┤ X ├──■──┤ X ├┤ H ├┤ X ├──■──┤ X ├┤ H ├─────┤M├───
     ├───┤└───┘  │  ├───┤├───┤├───┤┌─┴─┐├───┤├───┤┌───┐└╥┘┌─┐
q_1: ┤ H ├───────■──┤ H ├┤ X ├┤ H ├┤ X ├┤ H ├┤ X ├┤ H ├─╫─┤M├
     ├───┤┌───┐┌─┴─┐└───┘└───┘└───┘└───┘└───┘└───┘└───┘ ║ └╥┘
q_2: ┤ X ├┤ H ├┤ X ├────────────────────────────────────╫──╫─
     └───┘└───┘└───┘                                    ║  ║ 
c: 2/═══════════════════════════════════════════════════╩══╩═
                                                        0  1 
Measurement results (reversed):
{'01': 1000}


In [23]:
from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator
import math

def oracle(circuit, input_qubits, target_qubit, secret_element):
    """
    Oracle marks the secret element with a phase flip using X gates and multi-controlled Toffoli.
    """
    # Flip qubits corresponding to 0s
    for i, bit in enumerate(secret_element):
        if bit == '0':
            circuit.x(input_qubits[i])
    
    # Multi-controlled NOT (phase flip) on target
    circuit.mcx(input_qubits, target_qubit)
    
    # Revert flipped qubits
    for i, bit in enumerate(secret_element):
        if bit == '0':
            circuit.x(input_qubits[i])
    
    return circuit

def diffusion_operator(circuit, input_qubits):
    """
    Diffusion operator (inversion about the mean) generalized for any number of qubits.
    """
    num_qubits = len(input_qubits)
    # 1. Hadamard
    for q in input_qubits:
        circuit.h(q)
    # 2. X gates
    for q in input_qubits:
        circuit.x(q)
    # 3. Multi-controlled Z
    if num_qubits == 1:
        circuit.z(input_qubits[0])
    elif num_qubits == 2:
        circuit.h(input_qubits[1])
        circuit.cx(input_qubits[0], input_qubits[1])
        circuit.h(input_qubits[1])
    else:
        circuit.h(input_qubits[-1])
        circuit.mcx(input_qubits[:-1], input_qubits[-1])
        circuit.h(input_qubits[-1])
    # 4. X gates
    for q in input_qubits:
        circuit.x(q)
    # 5. Hadamard
    for q in input_qubits:
        circuit.h(q)
    
    return circuit

def grovers_algorithm(num_qubits, secret_element, shots=1024):
    """
    Generalized Grover's algorithm for any number of qubits and arbitrary secret element.
    Automatically computes optimal number of iterations.
    """
    circuit = QuantumCircuit(num_qubits + 1, num_qubits)  # extra target qubit
    input_qubits = list(range(num_qubits))
    target_qubit = num_qubits

    # Step 1: Equal superposition
    for q in input_qubits:
        circuit.h(q)
    
    # Step 2: Target qubit in |-> state
    circuit.x(target_qubit)
    circuit.h(target_qubit)
    
    # Step 3: Compute optimal number of Grover iterations
    N = 2 ** num_qubits
    m = int(math.floor((math.pi / 4) * math.sqrt(N)))  # optimal iterations

    # Step 4: Apply Grover iterations
    for _ in range(m):
        circuit = oracle(circuit, input_qubits, target_qubit, secret_element)
        circuit = diffusion_operator(circuit, input_qubits)
    
    # Step 5: Measurement
    for i, q in enumerate(input_qubits):
        circuit.measure(q, i)
    
    # Simulation
    sim = AerSimulator()
    result = sim.run(circuit, shots=shots).result()
    counts = result.get_counts()
    
    # Reverse bitstrings for readability
    counts_reversed = {k[::-1]: v for k, v in counts.items()}

    print("Grover's algorithm circuit:")
    print(circuit)
    print("Measurement results :")
    print(counts_reversed)

    return counts_reversed

# Example usage
if __name__ == "__main__":
    # Works for any number of qubits and secret element
    grovers_algorithm(num_qubits=4, secret_element='1011')


Grover's algorithm circuit:
     ┌───┐          ┌───┐┌───┐          ┌───┐┌───┐          ┌───┐┌───┐     »
q_0: ┤ H ├───────■──┤ H ├┤ X ├───────■──┤ X ├┤ H ├───────■──┤ H ├┤ X ├─────»
     ├───┤┌───┐  │  ├───┤├───┤┌───┐  │  ├───┤├───┤┌───┐  │  ├───┤├───┤┌───┐»
q_1: ┤ H ├┤ X ├──■──┤ X ├┤ H ├┤ X ├──■──┤ X ├┤ H ├┤ X ├──■──┤ X ├┤ H ├┤ X ├»
     ├───┤└───┘  │  ├───┤├───┤└───┘  │  ├───┤├───┤└───┘  │  ├───┤├───┤└───┘»
q_2: ┤ H ├───────■──┤ H ├┤ X ├───────■──┤ X ├┤ H ├───────■──┤ H ├┤ X ├─────»
     ├───┤       │  ├───┤├───┤┌───┐┌─┴─┐├───┤├───┤┌───┐  │  ├───┤├───┤┌───┐»
q_3: ┤ H ├───────■──┤ H ├┤ X ├┤ H ├┤ X ├┤ H ├┤ X ├┤ H ├──■──┤ H ├┤ X ├┤ H ├»
     ├───┤┌───┐┌─┴─┐└───┘└───┘└───┘└───┘└───┘└───┘└───┘┌─┴─┐└───┘└───┘└───┘»
q_4: ┤ X ├┤ H ├┤ X ├───────────────────────────────────┤ X ├───────────────»
     └───┘└───┘└───┘                                   └───┘               »
c: 4/══════════════════════════════════════════════════════════════════════»
                                                